In [1]:
import gmaps
from IPython.display import display
import ipywidgets as widgets
import googlemaps
from collections import defaultdict as dd
from datetime import datetime
from progiter import ProgIter as prog
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.path
from ipywidgets.embed import embed_minimal_html

with open("api_key", "r") as file:
    api_key = file.read()
gmaps.configure(api_key=api_key)
gmspy = googlemaps.Client(key=api_key)



In [2]:
class Loc(object):
    def __init__(self, name, short, lng=None, lat=None, color=(0,0,0), **kwargs):
        self.name = name
        self.short = short
        self.color= color
        
        if lng and lat:
            self.lng = lng
            self.lat = lat
        else:
            self.find_lt()

        self.directions = {}
    
    def __str__(self):
        return self.name
            
    def find_lt(self):
        self.loc = gmspy.geocode(self.name)[0]
        print("{}: {}".format(self.name, self.loc["formatted_address"]))
        self.lng = self.loc["geometry"]["location"]["lng"]
        self.lat = self.loc["geometry"]["location"]["lat"]
    
    def get_lt(self):
        return (self.lat, self.lng)

class Faculty(Loc):
    def __init__(self, *args, students=0, **kwargs):
        self.students = students
        self.color = "rgba(50, 50, 50, 0.5)"
        self.closest = None
        super().__init__(*args, **kwargs)

def get_circle(loc, radius, **kwargs):
    loc.plot = gmaps.Circle(
        radius=radius,
        center=(loc.lat, loc.lng),
        **kwargs
    )
    return loc.plot

def get_colored_circle(*args, color, fill_opacity=0.1, stroke_opacity=1, **kwargs):
    return get_circle(
        *args,
        fill_color=color,
        fill_opacity=fill_opacity,
        stroke_color=color,
        stroke_opacity=stroke_opacity
    )


def get_faculty_cicles(faculties, norm=20, **kwargs):
    objs = []
    for faculty in faculties.values():
        radius = faculty.students/norm
        if faculty.closest:
            objs.append(get_colored_circle(faculty, radius, color=faculty.closest.color))
        else:
            objs.append(get_circle(faculty, radius))
    return objs

def map_style(fig):
    with open("silver.json", "r") as file:
        style = file.read()
    fig.styles = style
        
def add_layer(fig, features):
    fig.add_layer(gmaps.drawing_layer(features=features, show_controls=False))
        


In [3]:
TU_coordinates = [
    (51.986816, 4.376383),
    (51.989374, 4.385949),
    (51.994826, 4.382106),
    (51.997765, 4.380045),
    (52.003343, 4.376126),
    (52.004744, 4.374916),
    (52.006242, 4.372797),
    (52.007045, 4.371525),
    (52.007711, 4.371606),
    (52.009137, 4.371519),
    (52.008769, 4.368881),
    (52.007560, 4.366079),
    (52.006328, 4.367199),
    (52.005263, 4.368880),
    (52.003696, 4.367351),
    (52.002253, 4.366597),
    (52.002054, 4.366924),
    (51.999733, 4.368465),
    (51.995577, 4.370787),
    (51.990907, 4.373798),
    (51.989850, 4.374313),
    (51.989296, 4.374803),
]
_stations = [
    ["SD",   "Station Delft",           (239,71,111)],
    ["DC",   "Delft Campus Station",    (6,214,160)]
]
stations = {short: Loc(name, short, color=color) for short, name, color in _stations}

Station Delft: Station Delft, 2611 AC Delft, Netherlands
Delft Campus Station: Delft Campus, 2623 AX Delft, Netherlands


In [4]:
TU_polygons = [gmaps.Polygon(TU_coordinates, fill_color=(0,166,214), stroke_color=(0,166,214), fill_opacity=0.1)]
NS_circles = [get_colored_circle(station, 50, color=station.color, fill_opacity=1) for station in stations.values()]

fig = gmaps.figure(center=(51.998487, 4.374864), zoom_level=15)
map_style(fig)
add_layer(fig, TU_polygons)
add_layer(fig, NS_circles)
embed_minimal_html('export/base_map.html', views=[fig])
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# search_region = [
#     (52.002590, 4.376638),
#     (52.000151, 4.378239),
#     (51.995654, 4.370645),
#     (51.998691, 4.369007),
# ]

lats, lngs = list(map(list, zip(*TU_coordinates)))
ypts, xpts = 60, 40
y, x = np.meshgrid(np.linspace(min(lats), max(lats), ypts), np.linspace(min(lngs), max(lngs), xpts)) 
y, x = y.flatten(), x.flatten()
points = np.vstack((y, x)).T 

p = matplotlib.path.Path(TU_coordinates)
grid = p.contains_points(points)
search_pts = [(round(lat, 6), round(lng, 6)) for lat, lng, in_region in zip(y, x, grid) if in_region]

print("search region contains {} points.".format(sum(grid)))


search region contains 1004 points.


In [6]:
# search_polygon = [gmaps.Polygon(search_region, fill_color=(255, 209, 102), stroke_color=(255, 209, 102), fill_opacity=0.1)]
search_symbols = gmaps.symbol_layer(search_pts, fill_color=(255, 209, 102), stroke_color=(255, 209, 102), scale=1)

fig = gmaps.figure(center=(51.998487, 4.374864), zoom_level=15)
map_style(fig)
add_layer(fig, TU_polygons)
add_layer(fig, NS_circles)
# add_layer(fig, search_polygon)
fig.add_layer(search_symbols)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
pt_closest = []
pt_array = []
for lat, lng, in_region in prog(zip(y, x, grid)):
    if in_region:
        times = dd(int)
        for station in stations.values():
            directions = gmspy.directions((lat, lng), (station.lat, station.lng), mode="bicycling", departure_time=datetime.now())
            times[station.short] = sum(leg["duration"]["value"] for leg in directions[0]["legs"])
        short = min(times, key=times.get)
        pt_closest.append(short)
        pt_array.append(short)
    else:
        pt_array.append(None)


 2400/?... rate=15.99 Hz, total=0:02:30, wall=15:55 CET


In [8]:
sd_locs = [pt for pt, station in zip(search_pts, pt_closest) if station == "SD"]
dc_locs = [pt for pt, station in zip(search_pts, pt_closest) if station == "DC"]

if len(sd_locs) > len(dc_locs):
    name = stations["SD"].name
    per = round(len(sd_locs)/len(search_pts)*100)
else:
    name = stations["DC"].name
    per = round(len(dc_locs)/len(search_pts)*100)

print("The winner is {} covering {}% of the TU Delft campus area.".format(name, per))

The winner is Delft Campus Station covering 65% of the TU Delft campus area.


In [9]:

sd_symbols = gmaps.symbol_layer(sd_locs, fill_color=stations["SD"].color, stroke_color=stations["SD"].color, scale=1)
dc_symbols = gmaps.symbol_layer(dc_locs, fill_color=stations["DC"].color, stroke_color=stations["DC"].color, scale=1)

fig = gmaps.figure(center=(51.998487, 4.374864), zoom_level=15)
map_style(fig)
add_layer(fig, TU_polygons)
add_layer(fig, NS_circles)
# add_layer(fig, search_polygon)
fig.add_layer(sd_symbols)
fig.add_layer(dc_symbols)

fig

Figure(layout=FigureLayout(height='420px'))

## Faculties


In [10]:
_faculties = [
    ["AS",      1186,   "TU Delft Faculty of Applied Sciences"],
    ["EEMCS",   4024,   "TU Delft Faculty Electrical Engineering, Mathematics and Computer Science"],
    ["CEG",     3644,   "TU Delft Faculty Of Civil Engineering and Geosciences"],
    ["AE",      2642,   "TU Delft Faculty of Aerospace Engineering"],
    ["ASN",     1958,   "TU Delft TNW Applied Sciences"],
    ["3ME",     4866,   "TU Delft Faculty of Mechanical, Maritime and Materials Engineering"],
    ["IDE",     1964,   "TU Delft Faculty of Industrial Design Engineering"],
    ["IH",      1236,   "Inholland Delft"],
    ["HH",      2658,   "De Haagse Hoogeschool (vesting Delft)"],
    ["BK",      2806,   "TU Delft Faculty of Architecture"],
    ["TPM",     1613,   "TU Delft Faculty of Technology, Policy and Management"]
]
faculties = {short: Faculty(name, short, students=students) for short, students, name in _faculties}

TU Delft Faculty of Applied Sciences: Lorentzweg 1, 2628 CJ Delft, Netherlands
TU Delft Faculty Electrical Engineering, Mathematics and Computer Science: Mekelweg 4, 2628 CD Delft, Netherlands
TU Delft Faculty Of Civil Engineering and Geosciences: Faculteit Civiele Techniek en Geowetenschappen (CITG), Stevinweg 1, 2628 CN Delft, Netherlands
TU Delft Faculty of Aerospace Engineering: Kluyverweg 1, 2629 HS Delft, Netherlands
TU Delft TNW Applied Sciences: Van der Maasweg 9, 2629 HZ Delft, Netherlands
TU Delft Faculty of Mechanical, Maritime and Materials Engineering: Mekelweg 2, 2628 CD Delft, Netherlands
TU Delft Faculty of Industrial Design Engineering: Landbergstraat 15, 2628 CE Delft, Netherlands
Inholland Delft: Rotterdamseweg 141, 2628 AL Delft, Netherlands
De Haagse Hoogeschool (vesting Delft): Rotterdamseweg 137, 2628 AL Delft, Netherlands
TU Delft Faculty of Architecture: Julianalaan 134, 2628 BL Delft, Netherlands
TU Delft Faculty of Technology, Policy and Management: Jaffalaan

In [12]:
fig = gmaps.figure(center=(51.998487, 4.374864), zoom_level=15)
map_style(fig)
add_layer(fig, TU_polygons)
add_layer(fig, NS_circles)
add_layer(fig, get_faculty_cicles(faculties))
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
station_students = dd(int)
for faculty in faculties.values():
    times = dd(int)
    for station in stations.values():
        directions = gmspy.directions(faculty.name, station.name, mode="bicycling", departure_time=datetime.now())
        faculty.directions[station.short] = directions
        times[station.short] = sum(leg["duration"]["value"] for leg in directions[0]["legs"])
    closest = min(times, key=times.get)
    faculty.closest = stations[closest]
    print("{} is closest to {}".format(faculty.name, stations[closest].name))
    station_students[closest] += faculty.students


TU Delft Faculty of Applied Sciences is closest to Delft Campus Station
TU Delft Faculty Electrical Engineering, Mathematics and Computer Science is closest to Delft Campus Station
TU Delft Faculty Of Civil Engineering and Geosciences is closest to Delft Campus Station
TU Delft Faculty of Aerospace Engineering is closest to Delft Campus Station
TU Delft TNW Applied Sciences is closest to Delft Campus Station
TU Delft Faculty of Mechanical, Maritime and Materials Engineering is closest to Station Delft
TU Delft Faculty of Industrial Design Engineering is closest to Station Delft
Inholland Delft is closest to Station Delft
De Haagse Hoogeschool (vesting Delft) is closest to Station Delft
TU Delft Faculty of Architecture is closest to Station Delft
TU Delft Faculty of Technology, Policy and Management is closest to Station Delft


In [14]:
winner = max(station_students, key=station_students.get)
loser = min(station_students, key=station_students.get)
print("The winner is {} with {} students over {}".format(stations[winner], station_students[winner], station_students[loser]))

The winner is Station Delft with 15143 students over 13454


In [15]:
fig2 = gmaps.figure(center=(51.998487, 4.374864), zoom_level=15)
map_style(fig2)
add_layer(fig2, TU_polygons)
add_layer(fig2, NS_circles)
add_layer(fig2, get_faculty_cicles(faculties))
fig2


Figure(layout=FigureLayout(height='420px'))